In [1]:
#Import libraries 

import json, requests
import time, datetime
import pandas as pd

# Viewing configs for notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : 'w'}
# Code below allows multiple console outputs to be generated without print statements
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
json_file = open('creds.json')
json_str = json_file.read()
json_data = json.loads(json_str)

In [3]:
json_data

{'APP_ID': 'MCsTZm6nyIiU66A4alks', 'APP_CODE': 'eedX5DhfpAJzo-30ovwlNA'}

In [4]:
json_data['APP_CODE']

'eedX5DhfpAJzo-30ovwlNA'

In [5]:
proximity_coords = "34.1643%2C-118.6919%2C23664"

In [6]:
url = f"https://traffic.api.here.com/traffic/6.0/incidents.json?prox={proximity_coords}&criticality=critical&app_id={json_data['APP_ID']}&app_code={json_data['APP_CODE']}"


#proximity 

In [7]:
res = requests.get(url)
date_requested = time.strftime('%Y-%m-%d')
time_requested = time.strftime('%H:%M') # 24-hour format
date_requested; time_requested

'2019-07-26'

'14:00'

In [8]:
res.status_code

200

In [12]:
 new_json = res.json()

In [13]:
print(new_json.keys())
new_json['TRAFFICITEMS'].keys()

#hmaps_json['TRAFFIC_ITEMS']['TRAFFIC_ITEM'];

dict_keys(['TIMESTAMP', 'VERSION', 'TRAFFICITEMS'])


dict_keys(['TRAFFICITEM'])

In [14]:
incident_dict = new_json['TRAFFICITEMS']['TRAFFICITEM']
incident_dict

[{'TRAFFICITEMID': 3550270994481212584,
  'ORIGINALTRAFFICITEMID': 1490074230888900810,
  'TRAFFICITEMSTATUSSHORTDESC': 'ACTIVE',
  'TRAFFICITEMTYPEDESC': 'CONSTRUCTION',
  'STARTTIME': '09/21/2018 16:00:00',
  'ENDTIME': '07/27/2019 01:03:25',
  'ENTRYTIME': '07/26/2019 20:05:01',
  'CRITICALITY': {'ID': '0', 'DESCRIPTION': 'critical'},
  'VERIFIED': True,
  'ABBREVIATION': {'SHORTDESC': 'CONST', 'DESCRIPTION': 'construction'},
  'LOCATION': {'INTERSECTION': {'ORIGIN': {'ID': '',
     'STREET1': {'ADDRESS1': 'Constellation Blvd'},
     'STREET2': {'ADDRESS1': 'CA-2'},
     'COUNTY': 'Los Angeles',
     'STATE': '',
     'PROXIMITY': {'ID': 'MID', 'DESCRIPTION': 'midway between'}},
    'TO': {'ID': '',
     'STREET1': {'ADDRESS1': 'Constellation Blvd'},
     'STREET2': {'ADDRESS1': 'Santa Monica Blvd'},
     'COUNTY': 'Los Angeles',
     'STATE': '',
     'PROXIMITY': {'ID': 'MID', 'DESCRIPTION': 'midway between'}}},
   'GEOLOC': {'ORIGIN': {'LATITUDE': 34.06034, 'LONGITUDE': -118.4135

In [15]:
malibu_incidents2 = pd.DataFrame(incident_dict)

In [16]:
malibu_incidents2.shape

(2, 13)

In [17]:
malibu_incidents2.isnull().sum()

ABBREVIATION                  0
CRITICALITY                   0
ENDTIME                       0
ENTRYTIME                     0
LOCATION                      0
ORIGINALTRAFFICITEMID         0
STARTTIME                     0
TRAFFICITEMDESCRIPTION        0
TRAFFICITEMDETAIL             0
TRAFFICITEMID                 0
TRAFFICITEMSTATUSSHORTDESC    0
TRAFFICITEMTYPEDESC           0
VERIFIED                      0
dtype: int64

In [18]:
locations = pd.DataFrame(list(malibu_incidents2['LOCATION']))

In [19]:
locations.head()

,GEOLOC,INTERSECTION,NAVTECH,POLITICALBOUNDARY
0,"{'ORIGIN': {'LATITUDE': 34.06034, 'LONGITUDE':...","{'ORIGIN': {'ID': '', 'STREET1': {'ADDRESS1': ...","{'EDGE': {'EDGEID': ['870233673', '870233672',...","{'METROAREA': {'value': '', 'ID': 6}, 'COUNTY'..."
1,"{'ORIGIN': {'LATITUDE': 34.05869, 'LONGITUDE':...","{'ORIGIN': {'ID': '', 'STREET1': {'ADDRESS1': ...","{'EDGE': {'EDGEID': ['1152045210', '1152045211...","{'METROAREA': {'value': '', 'ID': 6}, 'COUNTY'..."


In [24]:
intersections = pd.DataFrame(list(locations['INTERSECTION']))

In [25]:
intersections.head()

,ORIGIN,TO
0,"{'ID': '', 'STREET1': {'ADDRESS1': 'Constellat...","{'ID': '', 'STREET1': {'ADDRESS1': 'Constellat..."
1,"{'ID': '', 'STREET1': {'ADDRESS1': 'Constellat...","{'ID': '', 'STREET1': {'ADDRESS1': 'Constellat..."


In [30]:
intersections['ORIGIN'].apply(pd.Series)

,ID,STREET1,STREET2,COUNTY,STATE,PROXIMITY
0,,{'ADDRESS1': 'Constellation Blvd'},{'ADDRESS1': 'CA-2'},Los Angeles,,"{'ID': 'MID', 'DESCRIPTION': 'midway between'}"
1,,{'ADDRESS1': 'Constellation Blvd'},{'ADDRESS1': 'Santa Monica Blvd'},Los Angeles,,"{'ID': 'MID', 'DESCRIPTION': 'midway between'}"


In [58]:
geo_coor = pd.DataFrame(list(locations['GEOLOC']))

In [59]:
geo_coor

,ORIGIN,TO
0,"{'LATITUDE': 34.05869, 'LONGITUDE': -118.41584}","[{'LATITUDE': 34.06034, 'LONGITUDE': -118.41357}]"
1,"{'LATITUDE': 34.04012, 'LONGITUDE': -118.57928}","[{'LATITUDE': 34.082328, 'LONGITUDE': -118.598..."
2,"{'LATITUDE': 34.06034, 'LONGITUDE': -118.41357}","[{'LATITUDE': 34.05869, 'LONGITUDE': -118.41584}]"
3,"{'LATITUDE': 34.07898, 'LONGITUDE': -118.75556}","[{'LATITUDE': 34.066566, 'LONGITUDE': -118.754..."
4,"{'LATITUDE': 34.33766, 'LONGITUDE': -118.5144}","[{'LATITUDE': 34.33527, 'LONGITUDE': -118.51109}]"
5,"{'LATITUDE': 34.082328, 'LONGITUDE': -118.598688}","[{'LATITUDE': 34.04012, 'LONGITUDE': -118.57928}]"
6,"{'LATITUDE': 34.33527, 'LONGITUDE': -118.51109}","[{'LATITUDE': 34.33973, 'LONGITUDE': -118.50654}]"


In [62]:
geo_coor.keys()

Index(['ORIGIN', 'TO'], dtype='object')

In [67]:
origin = geo_coor['ORIGIN'].apply(pd.Series)
origin

,LATITUDE,LONGITUDE
0,34.058690,-118.415840
1,34.040120,-118.579280
2,34.060340,-118.413570
3,34.078980,-118.755560
4,34.337660,-118.514400
5,34.082328,-118.598688
6,34.335270,-118.511090


In [65]:
geo_coor['TO']

0    [{'LATITUDE': 34.06034, 'LONGITUDE': -118.41357}]
1    [{'LATITUDE': 34.082328, 'LONGITUDE': -118.598...
2    [{'LATITUDE': 34.05869, 'LONGITUDE': -118.41584}]
3    [{'LATITUDE': 34.066566, 'LONGITUDE': -118.754...
4    [{'LATITUDE': 34.33527, 'LONGITUDE': -118.51109}]
5    [{'LATITUDE': 34.04012, 'LONGITUDE': -118.57928}]
6    [{'LATITUDE': 34.33973, 'LONGITUDE': -118.50654}]
Name: TO, dtype: object

In [69]:
# Add a strat/end column and create an incident column thats the index 
origin['start_end'] = 'start'
origin['incident'] = origin.index
origin

,LATITUDE,LONGITUDE,start_end,incident
0,34.058690,-118.415840,start,0
1,34.040120,-118.579280,start,1
2,34.060340,-118.413570,start,2
3,34.078980,-118.755560,start,3
4,34.337660,-118.514400,start,4
5,34.082328,-118.598688,start,5
6,34.335270,-118.511090,start,6


In [73]:
to = pd.DataFrame(s for k, v in geo_coor['TO'].items() for s in v)

In [75]:
to['start_end'] = 'end'
to['incident'] = to.index
to

,LATITUDE,LONGITUDE,start_end,incident
0,34.060340,-118.413570,end,0
1,34.082328,-118.598688,end,1
2,34.058690,-118.415840,end,2
3,34.066566,-118.754557,end,3
4,34.335270,-118.511090,end,4
5,34.040120,-118.579280,end,5
6,34.339730,-118.506540,end,6


In [77]:
total_locations = pd.concat([origin, to])

In [78]:
total_locations.to_csv('live_here_locations.csv')